In [ ]:
import os
import shutil
import json
import glob
import docx
import torch
import numpy as np
import pandas as pd
import openai
from openai import OpenAI
from tqdm.notebook import tqdm
from docx import Document
from docxcompose.composer import Composer
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.shared import Pt
import markdown
import markdown2
import pdfkit
from pygments.formatters import HtmlFormatter
import pypandoc
from bs4 import BeautifulSoup
import re
# from weasyprint import HTML

In [ ]:
# # Path to the JSON file where you want to save the mapping
# json_file_path = "/Users/alaa/Documents/ucsf/data/addo_dc/gpt_data/filename_mapping.json"

# # Writing the dictionary to the JSON file
# with open(json_file_path, 'r') as json_file:
#     transcript_id_map = json.load(json_file)
# print(transcript_id_map)

In [ ]:
# def add_transcript_id_to_docs(file_id_map: dict, file_dir: str):
#     for original_filename, filename in transcript_id_map.items():
#         file_id = filename.split('_')[1].split('.')[0]
#         filepath = f"{file_dir}/{filename}"
#         # Open the document
#         doc = Document(filepath)
#         # Add the Transcript ID at the beginning
#         doc_paragraph = doc.paragraphs[0]
#         doc_paragraph.insert_paragraph_before(f"Transcript ID: {file_id}")
#         # Save the document with the updated content
#         doc.save(filepath)
# # add_transcript_id_to_docs(file_id_map=transcript_id_map, 
# #                           file_dir="/Users/alaa/Documents/ucsf/data/addo_dc/gpt_data")

In [ ]:
all_contents = []

In [ ]:
# List all application file and structure
all_application_entities = glob.glob("/Users/alaa/Documents/swiftcraft/code/swiftcraft_backend/**/*", recursive=True)
# Filter to only include files (exclude directories)
all_application_filepaths = [f for f in all_application_entities if not os.path.isdir(f) \
                             and 'README' not in f.split('/')[-1] \
                             and 'pyc' not in f.split('/')[-1] \
                             and 'table' not in f.split('/')[-1] \
                             and 'column' not in f.split('/')[-1] \
                             and 'extra' not in f.split('/')[-1] \
                            and 'zip' not in f.split('/')[-1]]
print(len(all_application_filepaths))

In [ ]:
# List all application file and structure
all_fe_entities = glob.glob("/Users/alaa/Documents/swiftcraft/code/swiftcraft_frontend/src/**/*.js", recursive=True)
# Filter to only include files (exclude directories)
all_fe_filepaths = [f for f in all_fe_entities if not os.path.isdir(f) \
                             and 'README' not in f.split('/')[-1] \
                             and 'pyc' not in f.split('/')[-1] \
                             and 'table' not in f.split('/')[-1] \
                             and 'column' not in f.split('/')[-1] \
                             and 'extra' not in f.split('/')[-1] \
                            and 'zip' not in f.split('/')[-1]]
print(len(all_fe_entities))

In [ ]:
agent_task = """You will assist in the development of a FastAPI application for SwiftCraft.
Here we provide each of the files that belong to the codebase. 
You are always expected to work off of the existing codebase, 
modifying appropriate existing files or creating new files only if needed."""

In [ ]:
# "/Users/alaa/Documents/swiftcraft/code/swiftcraft_backend/email_service_account/swiftcraftweb-21142c8c9e06.json"

In [ ]:
# Define the keywords
keywords = ["custom-timeouts", "schemas", "models", "database", 
            "agent_tech", "agent_mech", 
#             "email.py",
#             "email_service.py",
#             "check_active_projects"
#             "email_service.py"
            "communication",
            "communication_service",
            "sms_service",
            "sms",
#             "stakeholder",
#             "activation", 
#             "account",
#             "activation_service.py",
#             "project.py", 
#             "project_service.py",
#             "check_active_projects.py"
#             "instruction"
           ]

all_messages = [{"role": "user", "content": agent_task}]

for fp in all_application_filepaths:
    # Only proceed if the file path contains at least one keyword
    if any(keyword in fp for keyword in keywords):
        if 'outlook_email.py' in fp or 'gmail_email' in fp or 'outlook_email_service.py' in fp:
            continue
        filename = fp.split('/')[-1]
        print(filename)
        with open(fp, 'r') as file:
            file_content = file.read()
        message_content = f"file name: {filename}\n{file_content}"
        message = {"role": "user", "content": message_content}
        all_messages.append(message)

print(len(all_messages))

In [ ]:
# all_messages[6]

In [ ]:
# query_id = 9000

In [ ]:
# # all_messages = [{"role": "user", "content": agent_task}]
# for fp in all_fe_entities:
#     filename = fp.split('/')[-1]
#     print(filename)
#     try:
#         with open(fp, 'r') as file:
#             file_content = file.read()
#         message_content = f"file name: {filename}\n{file_content}"
#         message = {"role": "user", "content": message_content}
#         all_messages.append(message)
#     except:
#         continue

In [ ]:
query_id += 10
user_query = """
"""
user_message = {"role": "user", "content": user_query}


In [ ]:
# agent_message = {"role": "user", "content": raw_output}
# user_query2 = """I want the invite to be logged as a communication with a new thread and comm_type='calendar'
# """
# user_reply = {"role": "user", "content": user_query2}

In [ ]:
# err_msg = """
# Provide code for a python script that takes the markdown and replaces all newlines with appropriate newline character \n so that it can be copy
# """
# err_message = {"role": "user", "content": err_msg}

In [ ]:
agent_input = all_messages + [user_message]
#+ [agent_message] + [user_reply]
#+ [err_message]
agent_input

In [ ]:
client = OpenAI()
response = client.chat.completions.create(
    model="o1-preview",
#     reasoning_effort="high",
    messages=agent_input
)
print(response.choices[0].message.content)

In [ ]:
raw_output = response.choices[0].message.content

In [ ]:
with open(f'/docs/agent_output_test{query_id}_raw.md', 'w', encoding='utf-8') as f:
    f.write(raw_output)

In [ ]:
print(f'/docs/agent_output_test{query_id}_raw.md')